<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 19</h1>
</div>


# 1. Heap: Dangers of manual memory management
# 2. Ownership and borrowing in Rust
# 3. Methods in Rust


## Last time

* Possible data locations: stack and heap
* Mostly focused on the stack

## Heap management

**Memory allocation:**
* ask for a given amount of space 
* receives a pointer to it<br> (or an out of memory error)

**Freeing memory:**
* classical manual: explicitly return it
  * more complicated
* automatic: garbage collection
  * comes with additional costs



C: `malloc` / `free`

C++: `new` / `delete` + C

**Pitfalls of manual memory management:**

 * leaks: unused memory never returned

 * attempting to use a pointer to memory that was deallocated

 * returning memory that was already deallocated

<div align="center">
    <b>How does Rust deal with these problems?</b>
</div>

## Allocating on the heap in Rust

* Various methods. The simplest via `Box::new(...)`

In [55]:
// placing integers on the heap
let mut pointer = Box::new(2000);
let pointer2 : Box<i32> = Box::new(22);

In [56]:
// accessing data via a * operator
println!("vals: {} {}", *pointer, *pointer2);
println!("vals: {} {}", pointer, pointer2);
println!("vals: {} {}", &pointer, &pointer2);
println!("sum: {}", *pointer + *pointer2);
// println!("sum: {}", pointer + pointer2);


vals: 2000 22
vals: 2000 22
vals: 2000 22
sum: 2022


In [57]:
*pointer = 3000;
println!("sum: {}", *pointer + *pointer2);

sum: 3022


In [58]:
*pointer2 = 23;
println!("sum: {}", *pointer + *pointer2);

Error: cannot assign to `*pointer2`, as `pointer2` is not declared as mutable

In [61]:
println!("{}", pointer);
let z = drop(pointer);
println!("{:?}", z);
println!("{:?}", pointer);

Error: borrow of moved value: `pointer`

## Experiment with passing the pointer around

In [62]:
fn print_content(pointer:Box<i32>) {
    println!("content: {}", *pointer)
}

fn print_value(val: i32){
    println!("content: {}", val)
}

let p = Box::new(123);
print_value(*p);
print_value(*p);
print_content(p); 

content: 123
content: 123
content: 123


In [63]:
let q = Box::new(321);

print_content(q);
print_content(q);

Error: use of moved value: `q`

## What happened: Ownership

* Each value in Rust has a variable that is its **owner**
* Only **one** owner
* When the owner goes out of scope, the value is dropped

In [64]:
fn print_content(pointer:Box<i32>) {
    println!("content: {}", *pointer)
}

let q = Box::new(321);

print_content(q);
print_content(q);

Error: use of moved value: `q`

* First call to `print_content`: `Box::new(321)` is **moved** from `q` to `pointer`
* (if it compiled) at the end of `print_content`:
  * `Box::new(321)` would be dropped
  * its space on the heap deallocated

Second call can't proceed: **the content of `q` is gone**

## More examples of ownership

In [65]:
// won't work, value moved as well
let x = Box::new(123);
println!("x = {}",*x);
let y = x;
println!("x = {}",*x);

Error: borrow of moved value: `x`

Fix our previous example by returning the pointer

In [66]:
fn print_content(pointer:Box<i32>) -> Box<i32> {
    println!("content: {}", *pointer);
    pointer
}

let q = Box::new(321);

let q = print_content(q);
let q = print_content(q);
let q = print_content(q);

content: 321
content: 321
content: 321


## Avoiding moving values a lot: borrowing

In [67]:
#[derive(Debug)]
struct Road {
    intersection_1: u32,
    intersection_2: u32,
    max_speed: u32,
}

// adding a function in the namespace of Road
impl Road {
    // very useful constructor
    fn new(i1:u32,i2:u32,speed:u32) -> Road {
        Road {
            intersection_1: i1,
            intersection_2: i2,
            max_speed: speed,
        }
    }
}

let road = Road::new(13,23,25);
println!("{}",road.max_speed);

25


In [68]:
// checking whether it moves
let another = road;
println!("{}",road.max_speed);

Error: borrow of moved value: `road`

In [69]:
fn display_1(r:Road) {
    println!("{:?}",r);
}

let road = Road::new(101,102,30);
display_1(road);
// Uncomment this one to see ownership behavior
// display_1(road);  

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


In [70]:
// This one works due to returning the value
fn display_1(r:Road) -> Road {
    println!("{:?}",r);
    r
}

let mut road = Road::new(101,102,30);
road = display_1(road);
road = display_1(road);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


## Avoiding moving values a lot: borrowing

Read–only reference:
* Reference type becomes `&Type`
* To create: `&value`
* To access content: `*reference`

In [76]:
fn display_2(r:&Road) {
    println!("{:?}", *r); // This is the right one
    println!("{:?}", r); // In this case the compiler does the conversion for you but it may not work all times.
    println!("{:?}", &r); // Ditto here.
}

let road = Road::new(101,102,30);
display_2(&road); // <- have to explicitly create a reference
display_2(&road);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


## Avoiding moving values a lot: borrowing
Mutable reference:
* Reference type becomes `&mut Type`
* To create: `&mut value`
* To access content: `*reference`


In [72]:
// regular references won't work
fn update_speed(r:&Road, new_speed: u32) {
    // r.max_speed equivalent to (*r).max_speed
    // because Rust is smart
    r.max_speed = new_speed;
} 

Error: cannot assign to `r.max_speed`, which is behind a `&` reference

In [73]:
fn update_speed(r:&mut Road, new_speed: u32) {
    // r.max_speed equivalent to (*r).max_speed
    // because Rust is smart
    r.max_speed = new_speed;
} 

In [74]:
let mut road = Road::new(100,200,30);
display_2(&road);
update_speed(&mut road, 25);
display_2(&road);

Road { intersection_1: 100, intersection_2: 200, max_speed: 30 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 25 }


## Methods


* We can add functions that are directly associated with structs and enums!
  * Then we could call them: `road.display()` or `road.update_speed(25)`
* How?
  * Put them in the namespace of the type
  * make `self` the first argument

In [16]:
impl Road {
    
    // note &self: immutable reference
    fn display(&self) {
        println!("{:?}",*self);
    }
}

In [18]:
let mut road = Road::new(1,2,35);
road.display();
&road.display();
(&road).display();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }


## Methods (continued)

In [19]:
impl Road {
    fn update_speed(&mut self, new_speed:u32) {
        self.max_speed = new_speed;
    }
}

In [20]:
road.display();
road.update_speed(25);
road.display();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 25 }


In [23]:
impl Road {
    
    fn this_will_move(self) -> Road {
        self
    }
    
    fn this_will_not_move(&self) -> &Road {
        self
    }
}

In [26]:
fn testme() {
  let r = Road::new(1,2,35);
  let r3 = r.this_will_not_move();
  // let r2 = r.this_will_move();
  r.display();
  r3.display();
}
testme();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }


## Methods (summary)

* Make first parameter `self`
* Various options:
  - `self`: move will occur
  - `&self`: self will be immutable reference
  - `&mut self`: self will be mutable reference

## A few more pointer types

In [45]:
// A few more pointer types for future reference

use std::rc::Rc;
use std::cell::RefCell;

// A reference counted pointer.  Behaves like a box pointer in this example
let mut pointer = Rc::new(12);
let pointer2 : Rc<i32> = Rc::new(22);
println!("sum: {}", *pointer + *pointer2);

// But as we saw you can assigned box pointers to each other without clone
let mut pointer = Box::new(10);
let mut pointer2 = pointer.clone();
*pointer2 = 22;
println!("Box Vals {} {} ", *pointer, *pointer2);


// But Rc pointers with refcells allow for this, 
// though you have to tell the compiler a lot of information to allow it
let mut pointer = Rc::new(RefCell::new(10));
let mut pointer2 = Rc::new(Rc::clone(&pointer));
*rpointer.borrow_mut() = 22;
println!("Ref Vals {} {} ", *pointer.borrow(), *pointer2.borrow());

sum: 34
Box Vals 10 22 
Ref Vals 10 10 


## Next time

Additional topics related to what was covered today:
* Specifying type to be always copied
* Having multiple references at the same time

## Read chapter 4